In [1]:
import numpy as np
import pandas as pd

In [2]:
file1 = "data/taur-events.csv"
file2 = "data/taur-otu-table-15tpts.csv"

In [3]:
f1 = pd.read_csv(file1)
f2 = pd.read_csv(file2, header=None)

In [4]:
data1 = f1.to_numpy()
data2 = f2.to_numpy()

In [5]:
pid = np.array(f1['patientID'])

# obs

In [6]:
measure_pid = f2.loc[0:0, 1:]
measure_pid = np.array(measure_pid, dtype=int)
measure_pid = measure_pid.reshape((-1))
f2pid = np.unique(measure_pid)

In [7]:
print("{} patients with id {}".format(len(f2pid), f2pid))

21 patients with id [ 26  31  97 447 515 530 531 537 551 642 694 697 698 704 708 721 747 753
 763 765 769]


In [8]:
count = data2[2:-1, 1:].T.astype(int)
print("count shape (measures, microbio speicies):", count.shape)
count_sum_across_spieces = np.sum(count, axis=1, keepdims=True)
percentage = count / count_sum_across_spieces

count shape (measures, microbio speicies): (405, 770)


In [9]:
percentage_sum_across_measures = np.sum(percentage, axis=0)
spieces_name = data2[2:-1, 0]
k = 10
top_k_spieces_idx = np.argsort(percentage_sum_across_measures)[-k:][::-1]
print("top_k_spieces_idx:", top_k_spieces_idx)
for i, idx in enumerate(top_k_spieces_idx):
    print("top {:>2}, idx {:>4}, percentage_sum {:>5.2f}, name {}".format(i, idx, percentage_sum_across_measures[idx], spieces_name[idx]))

top_k_spieces_idx: [281 291 411 326 284 441 731  15  25 261]
top  0, idx  281, percentage_sum 98.14, name Bacteria;Firmicutes;Bacilli;Lactobacillales;Enterococcaceae;Enterococcus
top  1, idx  291, percentage_sum 60.05, name Bacteria;Firmicutes;Bacilli;Lactobacillales;Streptococcaceae;Streptococcus
top  2, idx  411, percentage_sum 34.52, name Bacteria;Firmicutes;Erysipelotrichia;Erysipelotrichales;Erysipelotrichaceae;Erysipelatoclostridium
top  3, idx  326, percentage_sum 31.39, name Bacteria;Firmicutes;Clostridia;Clostridiales;Lachnospiraceae;Blautia
top  4, idx  284, percentage_sum 18.89, name Bacteria;Firmicutes;Bacilli;Lactobacillales;Lactobacillaceae;Lactobacillus
top  5, idx  441, percentage_sum 12.88, name Bacteria;Firmicutes;Negativicutes;Veillonellales;Veillonellaceae;Veillonella
top  6, idx  731, percentage_sum 12.48, name Bacteria;Verrucomicrobia;Verrucomicrobiae;Verrucomicrobiales;Akkermansiaceae;Akkermansia
top  7, idx   15, percentage_sum 11.00, name Bacteria;Actinobacteri

In [10]:
dates = data2[1, 1:].astype(int)
count_obs = np.zeros((count.shape[0], k + 2)) 
count_obs[:, 0] = dates
count_obs[:, 1:-1] = count[:, top_k_spieces_idx]
count_obs[:, -1] = count.sum(axis=-1) - count[:,top_k_spieces_idx].sum(axis=-1)

In [11]:
obs = []

pid_measure_start = 0
for pid in f2pid:
    patient_obs = []
    for i in range(pid_measure_start, len(measure_pid)):
        if measure_pid[i] != pid:
            pid_measure_start = i
            break
        patient_obs.append(count_obs[i])
    obs.append(np.asarray(patient_obs))
print(len(obs), obs[0].shape)

21 (20, 12)


In [12]:
percentage_0 = obs[0][:,1:] / (obs[0][:,1:].sum(axis=-1)[:,None])

# Inputs

In [13]:
event_pid, event, event_start, event_end = data1.T
event_pid = np.array(event_pid, dtype=int)
event_start = np.array(event_start, dtype=int)
event_end = np.array(event_end, dtype=int)
unique_event = list(np.unique(event))
num_event = len(unique_event)
print("total {} kinds of events: {}".format(num_event, unique_event))

total 15 kinds of events: ['PCP prophylaxis agents', 'aminoglycosides', 'anti-VRE agents', 'anti-anaerobic agent', 'beta-lactamase inhibitors', 'carbapenems', 'first/second generation cephalosporins', 'fourth/fifth generation cephalosporins', 'glycopeptide', 'monobactams', 'penicillins', 'quinolones', 'surgery', 'tetracyclines', 'third generation cephalosporins']


In [14]:
Input = []
for pid in f2pid:
    patient_event_idxs = np.where(event_pid == pid)[0]
    input_start = min(event_start[patient_event_idxs])
    input_end = max(event_end[patient_event_idxs])
    input_len = input_end - input_start + 1
    patient_input = np.zeros((input_len, num_event + 1))
    patient_input[:, 0] = np.arange(input_start, input_end + 1)
    for event_idx in patient_event_idxs:
        patient_event = event[event_idx]
        event_id = unique_event.index(patient_event) + 1
        event_start_idx = event_start[event_idx] - input_start
        event_end_idx = event_end[event_idx] - input_start + 1
        patient_input[event_start_idx:event_end_idx, event_id] = np.ones(event_end_idx - event_start_idx)
    Input.append(patient_input)
print(len(Input), Input[0].shape)

21 (42, 16)


# counts

In [15]:
len(obs)

21

In [16]:
counts = []
for single_obs in obs:
    single_counts = single_obs.sum(axis=-1)
    counts.append(single_counts)

In [17]:
counts[1].shape

(26,)

In [18]:
obs[1].shape

(26, 12)

# split into train & test and save

In [19]:
import os, pickle
n_train = 17
data = {}
data["Ytrain"] = obs[:n_train]
data["Ytest"] = obs[n_train:]
data["Vtrain"] = Input[:n_train]
data["Vtest"] = Input[n_train:]
data["counts_train"] = counts[:n_train]
data["counts_test"] = counts[n_train:]
with open(os.path.join("data", "count_microbio.p"), "wb") as f:
    pickle.dump(data, f)

In [20]:
data.keys()

dict_keys(['Ytrain', 'Ytest', 'Vtrain', 'Vtest', 'counts_train', 'counts_test'])

# test runner

In [21]:
from src.utils.data_loader import load_data
from src.utils.data_interpolation import interpolate_data

In [22]:
datadir = "data/count_microbio.p"

In [23]:
Dx = 3

In [24]:
hidden_train, hidden_test, obs_train, obs_test, input_train, input_test, extra_inputs_train, extra_inputs_test = \
                load_data(datadir, Dx, False)
            

In [25]:
hidden_train, hidden_test, obs_train, obs_test, input_train, input_test, _mask_train, _mask_test, time_interval_train, time_interval_test, extra_inputs_train, extra_inputs_test = \
                interpolate_data(hidden_train, hidden_test, obs_train, obs_test, input_train, input_test,
                                 extra_inputs_train, extra_inputs_test, False)

In [26]:
extra_inputs_test

[array([119211., 119211.,  57969.,  57969.,  79237.,  58922.,  27552.,
         27552.,  27552.,  27552.,  27552.,  27552.,  27552.,  27552.,
         27552.,  27552.,  14404.,  14404.,  14787.,  14787.,  14787.,
         14787.,  14787.,  14787.,  13289.,   6998.,  29619.,  29619.,
         29619.,  12209.,  11605.,  23650.,  23650.,  10416.,  10416.,
         10416.,  10416.,  12408.]),
 array([247339.,  79964.,  57209.,  57209.,  38596.,   4069.,   4069.,
          4069.,  33809.,  92374., 109020.,  49130.,  89261.,  89261.,
         89261.,  45048.,  35792.,  31792.,  28567.,  63525.,  63525.,
         46655.,  60417.,  77563.,  42756.,  26556.,  33295.]),
 array([388758., 344191., 344191., 197936., 181335., 213836., 134485.,
        193135., 189283., 457265., 187615., 187615., 187615., 213900.,
        226571., 226571., 226571., 178110., 148345., 116465.,  82242.,
         56388., 214796., 144261., 236219.]),
 array([41748., 41748., 41748.,  9356., 23468., 16079., 39699., 13029.,


In [32]:
obs[-4][:,0]

array([-5., -3., -1.,  0.,  1., 11., 13., 19., 20., 21., 24., 25., 26.,
       28., 32.])

In [30]:
data["counts_test"][0].shape

(15,)

In [27]:
data["counts_test"]

[array([119211.,  57969.,  79237.,  58922.,  27552.,  14404.,  14787.,
         13289.,   6998.,  29619.,  12209.,  11605.,  23650.,  10416.,
         12408.]),
 array([247339.,  79964.,  57209.,  38596.,   4069.,  33809.,  92374.,
        109020.,  49130.,  89261.,  45048.,  35792.,  31792.,  28567.,
         63525.,  46655.,  60417.,  77563.,  42756.,  26556.,  33295.]),
 array([388758., 344191., 197936., 181335., 213836., 134485., 193135.,
        189283., 457265., 187615., 213900., 226571., 178110., 148345.,
        116465.,  82242.,  56388., 214796., 144261., 236219.]),
 array([41748.,  9356., 23468., 16079., 39699., 13029.,  4462., 14688.,
        12314., 23896., 31665., 22174., 18779., 20264., 10041., 19538.])]

In [33]:
extra_inputs_train[0].shape

(42,)

In [34]:
extra_inputs_train[1].shape

(47,)

In [35]:
obs_train[0].shape

(42, 11)

In [36]:
obs_train[1].shape

(47, 11)

In [38]:
for obs, extra_inputs in zip(obs_train, extra_inputs_train):
    assert obs.shape[0] == extra_inputs.shape[0]